## 날짜별로 여러 파일 이름으로 저장되어 있음
- 2020.01-2021.09 : 등기소포(창구, 택배)
- 2021.10-2023.10 : 방문소포, 등기소포
- 단) 저장되어 있는 폴더가 다르므로 잘 확인할 것

In [ ]:
import requests
from bs4 import BeautifulSoup

# 1. Pagenation
for page in range(0, 10):
    url="https://www.koreapost.go.kr/user/bbs/kpost/114/4255/bbsDataList.do?page={}&column=&search=&searchSDate=&searchEDate=&bbsDataCategory=".format(page)
    res=requests.get(url)
    soup=BeautifulSoup(res.text, "html.parser")
    titles=soup.findAll("a", "list_bbsDataTitle")
    # 2. 해당 페이지의 제목에 대해(월별 시도별 우편분류 접수 및 배달정보)
    for title in titles:
        href=title.attrs['href']
        titlestrip=title.text.strip()

        res2=requests.get("https://www.koreapost.go.kr"+title.attrs['href'])
        soup2=BeautifulSoup(res2.text, "html.parser")
        downlists=soup2.findAll("ul", "bbsFileList")[0].findAll("li")

        for downlist in downlists:
            downname=downlist.findAll("a")[0].text.strip()
            # 3. 등기소포, 방문소포에 해당하는 것만 다운로드
            if downname[0:11] in ["우정사업본부_등기소포", "우정사업본부_방문소포"] or downname[0:13] in ["우정사업본부_시도별 등기", "우정사업본부_시도별 방문"] or downname[3:16] in ["우정사업본부_시도별 등기", "우정사업본부_시도별 방문"] or downname[:13] in ["우정사업본부_우편물류정보"] or downname[0:14] in ["우편물류 접수 및 배달정보"]:
                href2=downlist.findAll("a")[0].attrs['href']
                downloadurl="https://www.koreapost.go.kr"+href2
                print(downname)
                local_filepath = "D:/programmers-devcourse/project2/우정사업본부/"+downname
                response = requests.get(downloadurl)

                # 요청이 성공적으로 이루어졌는지 확인(상태 코드 200).
                if response.status_code == 200:
                    # 로컬 파일을 이진 쓰기 모드로 열고 응답의 내용을 파일에 씁니다.
                    with open(local_filepath, 'wb') as file:
                        file.write(response.content)
                    print("파일 다운로드 성공.")
                else:
                    print(f"파일 다운로드 실패. 상태 코드: {response.status_code}")

## zip 폴더를 모두 압축풀기
- 한글이 깨지는 문제가 발생하여 patool 라이브러리를 사용

In [ ]:
!pip install patool

In [ ]:
import os
import patoolib

In [ ]:
def get_all_zip_filenames(folder_path):
    # folder_path: zip 파일 이름을 얻고자 하는 폴더의 경로

    # 폴더 내의 모든 파일 리스트
    all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # 확장자가 ".zip"인 파일들만 필터링
    zip_files = [f for f in all_files if f.lower().endswith('.zip')]

    return zip_files

In [ ]:
def unzip_all_folders(base_folder, output_folder):
    # base_folder: 압축을 해제할 폴더의 경로

    # zip 파일 이름을 얻고자 하는 폴더의 경로를 지정
    folder_path = 'D:/programmers-devcourse/project2/우정사업본부/'
    # base_folder 내의 모든 폴더 리스트
    folders = get_all_zip_filenames(folder_path)

    # 각 폴더에 대해 압축을 해제
    for folder in folders:
        zip_file = os.path.join(base_folder, folder)
        output_folder_full = os.path.join(output_folder, folder)

        # CP949로 명시적으로 지정하여 디코딩
        patoolib.extract_archive(zip_file, outdir=output_folder_full, verbosity=-1, interactive=False)

# 압축을 해제할 기본 폴더 경로를 지정
base_folder_path = 'D:/programmers-devcourse/project2/우정사업본부/'
output_folder_path = 'D:/programmers-devcourse/project2/우정사업본부-압축해제/'

# 함수 호출
unzip_all_folders(base_folder_path, output_folder_path)


## 데이터 전처리 수행
- 폴더이름 중 '창구, 택배, 방문, 등기' 문구가 포함되어 있지 않으면 폴더 하위에서 위의 문구가 포함된 폴더를 찾아 base_folder_path에 위치
- 연월별로 '창구, 택배, 방문, 등기' 문구가 포함되어 있는 2개의 폴더를 남기고 나머지는 삭제
- 특이하게 일부 지역의 경우 중량(g), 중량 // 부피(cm), 부피혼용해서 사용(전처리 필요)
- 연도월별로 모든지역을 통합해서 하나의 csv파일로 저장

In [ ]:
def get_all_subfolders(folder_path):
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]
    return subfolders

def get_all_files(folder_path):
    all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return all_files

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# 압축을 해제할 기본 폴더 경로를 지정
base_folder_path = 'D:/programmers-devcourse/project2/우정사업본부-압축해제/'
output_folder_path = 'D:/programmers-devcourse/project2/우정사업본부-전처리/'

folder_name_list=get_all_subfolders(base_folder_path)
for folder_name in folder_name_list:
    output_file_name=""
    date=""
    if "창구" in folder_name:
        output_file_name="Registered(counter)_Information"
    elif "택배" in folder_name:
        output_file_name="Registered(delivery)_Information"
    elif "방문" in folder_name:
        output_file_name="Visit_Information"
    elif "등기" in folder_name:
        output_file_name="Registered_Information"
    else:
        print("ERRRRRRRR")
        print(folder_name)
        pass
    file_list=get_all_files(folder_name)
    try:
        result=pd.read_csv(folder_name+"/"+file_list[0], encoding='utf-8')
    except UnicodeDecodeError:
        try:
            # 다른 인코딩 시도 (예: cp1252)
            result=pd.read_csv(folder_name+"/"+file_list[0], encoding='cp949')
        except UnicodeDecodeError:
            # 여러 가지 인코딩을 시도할 수도 있습니다.
            print(f"Failed to read file: {file}")

    result.dropna(inplace=True)
    if result.columns[-1]=='부피(cm)':
        result.rename(columns={'중량(g)':'중량','부피(cm)':'부피'}, inplace=True)
    print(result.columns)
    print(folder_name)
    for file in tqdm(file_list[1:], desc="파일 처리 중", leave=False):
        try:
            csv = pd.read_csv(os.path.join(folder_name, file), encoding='utf-8')
        except UnicodeDecodeError:
            try:
                # 다른 인코딩 시도
                csv = pd.read_csv(os.path.join(folder_name, file),encoding='cp949')
            except UnicodeDecodeError:
                # 여러 가지 인코딩을 시도할 수도 있습니다.
                print(f"Failed to read file: {file}")
        if csv.columns[-1]=='부피(cm)':
            csv.rename(columns={'중량(g)':'중량','부피(cm)':'부피'}, inplace=True)
        csv.dropna(inplace=True)
        result = pd.concat([result, csv], ignore_index=True)
    result_agg=result.groupby(['접수일자', '접수지역', '배달지역', '내용품']).agg({'중량': ['count', 'sum'], '부피':'sum'})
    result_agg=result_agg.reset_index().sort_values(by=['접수일자', '접수지역', '배달지역', '내용품'], ascending=True)
    result_agg.columns = [' '.join(col).strip() for col in result_agg.columns.values]
    date+=str(result_agg.loc[0, '접수일자'][0:4])+str(result_agg.loc[0, '접수일자'][5:7])
    result_agg.rename(columns={'접수일자':'receipt_date','접수지역': 'departure_area', '배달지역': 'destination_area', "내용품":"category", "중량 sum":"total_weight", "부피 sum":"total_volume", "중량 count":"total_deliveries"}, inplace=True)
    print("------------------------")
    print(output_folder_path+date+output_file_name)
    result_agg.to_csv(output_folder_path+date+output_file_name+'.csv')

모든 파일 하나로 저장